In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import requests
import shutil
import bs4
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import pyautogui
import cv2
import time

In [ ]:
#import excel that contains tiktok link
df_link = pd.read_excel('link tiktok.xlsx')

In [ ]:
PATH = "C:\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(PATH)

#empty list
link = []
judul = []
username = []
likes = []
views = []
comment = []
share = []

#ambil url yang ingin di scrape
for i in range(len(df_link.iloc[:,0])):
    link = df_link.iloc[i,0]

    #open and maximize window
    driver.get(link)
    driver.maximize_window()

    time.sleep(6)

    #CAPTCHA SOLVER
    #download image function
    def down(image, xpath):
        #get link image
        captcha = driver.find_element (By.XPATH, xpath)
        src = captcha.get_attribute('src')

        #download image
        response = requests.get(src, stream=True)
        with open(image, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response

    #download captcha image
    down('captcha_img.jpeg', "//*[@id='captcha-verify-image']")
    #download piece of captcha image
    down('captcha_piece.jpeg', "//*[@id='tiktok-verify-ele']/div/div[2]/img[2]")

    time.sleep(2)

    #read img
    img = cv2.imread('captcha_img.jpeg')
    img_piece = cv2.imread('captcha_piece.jpeg')

    #CONTOURS FUNCTION
    def bin_img(image):
        img = cv2.imread(image)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        binary_img = cv2.threshold(img_gray, 242, 255, cv2.THRESH_BINARY_INV)[1]

        return binary_img

    capt = bin_img('captcha_img.jpeg')
    piece = bin_img('captcha_piece.jpeg')

    #matching shape
    match = cv2.matchTemplate(capt, piece, cv2.TM_CCOEFF_NORMED)

    #check x, y in maximum value
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(match)

    #width and height
    w = piece.shape[1]
    h = piece.shape[0]

    time.sleep(2)

    #drag slider with mouse movement
    #gerakin mouse ke slider button
    pyautogui.moveTo(782, 722, duration=1.5)
    #drag mouse ke puzzle pece yang benar
    pyautogui.dragRel(0.78*max_loc[0], 0, button='left', duration=2)

    time.sleep(5)

    #exit login bar
    ext_login_bar = driver.find_element(By.XPATH, '//*[@id="login-modal"]/div[2]')
    ext_login_bar.click()

    time.sleep(2)

    #open profile
    profile = driver.find_element(By.XPATH, '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]')
    profile.click()

    time.sleep(2)
    
    #scroll to the bottom
    previous_height = driver.execute_script('return document.body.scrollHeight')

    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(3)
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if new_height == previous_height:
            break
        previous_height = new_height

    #scroll to the top
    driver.execute_script('window.scrollTo(0, 0)')

    #take the html source
    page_html = driver.page_source
    page_soup = bs4.BeautifulSoup(page_html, 'html.parser')
    #find all content
    content = page_soup.find_all('div', {'class':"tiktok-x6y88p-DivItemContainerV2 e19c29qe7"})

    len_content = len(content)

    #looping didalem profile mereka sampai ketemu link yang sama
    for j in range(1, len_content+1):
        xpath = '//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[%s]' %j
        xpath_url = '//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[%s]/div[1]/div/div/a' %j
        
        #grab the url of the content
        url = driver.find_element(By.XPATH, xpath_url)
        get_url = xpath.get_attribute('href')

        if get_url != i:
            continue
        else
        

#Result
df_result = pd.DataFrame(list(zip(link, judul, username, likes, views, comment, share)), columns=['link'. 'judul', 'username', 'likes', 'views', 'comment', 'share'])

#exit chrome
#driver.quit()

In [ ]:
#export data to excel
df_result.to_excel('tiktok data scrape result.xlsx', index=False)